<a href="https://colab.research.google.com/github/jainstuti/Tweets_classification/blob/main/rf_TMLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [167]:
#importing libraries
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [168]:
#importing dataset
dataset = pd.read_csv('Twitter_Data.csv')

In [169]:
dataset=dataset.head(90000)

In [170]:
dataset.shape

(90000, 2)

In [171]:
# to get the values in categorical data and their counts in the data
dataset['category'].value_counts()

 1.0    40059
 0.0    30375
-1.0    19566
Name: category, dtype: int64

In [172]:
dataset.describe()

,category
count,90000.000000
mean,0.227700
std,0.781447
min,-1.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [173]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   clean_text  89999 non-null  object 
 1   category    90000 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [174]:
dataset.isnull().sum()


clean_text    1
category      0
dtype: int64

In [175]:
is_NaN = dataset.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dataset[row_has_NaN]

print(rows_with_NaN)

    clean_text  category
148        NaN       0.0


In [176]:
dataset=dataset.dropna()

In [177]:
dataset = dataset.reset_index()

In [178]:
dataset.shape

(89999, 3)

In [179]:
dataset.head()

,index,clean_text,category
0,0,when modi promised “minimum government maximum...,-1.0
1,1,talk all the nonsense and continue all the dra...,0.0
2,2,what did just say vote for modi welcome bjp t...,1.0
3,3,asking his supporters prefix chowkidar their n...,1.0
4,4,answer who among these the most powerful world...,1.0


In [180]:
dataset =dataset.drop(['index'], axis = 1)

In [181]:
#cleaning the text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, len(dataset)):
  tweet = re.sub("[^a-zA-Z]", " ", dataset['clean_text'][i])
  tweet = tweet.lower()
  tweet = tweet.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  tweet = [ps.stem(word) for word in tweet if not word in set(all_stopwords)]
  tweet = ' '.join(tweet)
  corpus.append(tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [182]:
#creating bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

In [183]:
X.shape

(89999, 3000)

In [184]:
#slpiiting data into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [185]:
#randon forest
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [186]:
y_pred = classifier.predict(X_test)

In [187]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2512  647  746]
 [ 251 5437  384]
 [ 562  943 6518]]


0.8037222222222222

In [188]:
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)
print(np.mean(recall))
print(np.mean(precision))

0.7837045883839342
0.7938225249135189


In [189]:
f1Score = 2 * precision * recall / (precision + recall)
print(np.mean(f1Score))

0.7856257194823729
